# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


#Maps config
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
File=os.path.join('..','WeatherPy','OutputData','cities.csv')
Cities=pd.read_csv(File)
Cities.head()

,City,Cloudiness (%),Country,Date (UTC),Humidity (%),Lat,Lng,Max Tem (F),Wind Speed (mph)
0,sao joaquim,57,BR,1603677066,100,-28.29,-49.93,53.96,3.78
1,sur,0,OM,1603676975,64,22.57,59.53,72.27,5.68
2,cape town,1,ZA,1603676783,89,-33.93,18.42,61.00,4.00
3,upernavik,55,GL,1603677067,78,72.79,-56.15,13.96,5.75
4,hithadhoo,10,MV,1603677028,65,-0.60,73.08,83.70,6.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
Humidity=Cities['Humidity (%)']
MaxHumidity=float(Humidity.max())
Coordinates=Cities[['Lat','Lng']]
PointRadius=10


In [4]:
figure_layout={'width':'500px', 'height':'500px','margin':'0 auto 0 auto'}
fig=gmaps.figure(layout=figure_layout)
heatmap_layer=gmaps.heatmap_layer(
    Coordinates, weights=Humidity,
    max_intensity=MaxHumidity, point_radius=PointRadius
)

fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='500px', margin='0 auto 0 auto', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Criteria to Narrow for temperature, wind and cloudiness
#Ideal weather condition
MaxTemp=80
MinTem=70
WindSpeed=10
Cloudiness=0

narrowed_city_df=Cities.loc[
    (Cities['Max Tem (F)']>MinTem) &
    (Cities['Max Tem (F)']<MaxTemp) &
    (Cities['Wind Speed (mph)']<WindSpeed) &
    (Cities['Cloudiness (%)']==Cloudiness) ]


narrowed_city_df.head()


,City,Cloudiness (%),Country,Date (UTC),Humidity (%),Lat,Lng,Max Tem (F),Wind Speed (mph)
1,sur,0,OM,1603676975,64,22.57,59.53,72.27,5.68
15,bhinmal,0,IN,1603677070,52,25.00,72.25,73.26,2.17
95,doha,0,QA,1603677074,74,25.29,51.53,78.80,2.24
138,busselton,0,AU,1603677194,52,-33.65,115.33,73.99,3.00
169,dakar,0,SN,1603677250,88,14.69,-17.44,78.80,8.05


In [6]:
len(narrowed_city_df)

19

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
narrowed_city_df['Hotel Name']=''

target_radius=5000
target_type='hotel'
keyword='hotel'

for index,row in narrowed_city_df.iterrows():
    
    latitud=row['Lat']
    longitud=row['Lng']
    
    #set up a parameters dictionary
    params={'location':f'{latitud},{longitud}',
            'radius':target_radius,
            'type':target_type,
            'keyword':keyword,
            'key':g_key
        }
    #base url
    base_url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    try:
        narrowed_city_df.loc[index, 'Hotel Name']=response['results'][0]['name']
        
    except (KeyError, IndexError):
        narrowed_city_df.loc[index, 'Hotel Name']=np.nan

In [12]:
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df.head()

,City,Cloudiness (%),Country,Date (UTC),Humidity (%),Lat,Lng,Max Tem (F),Wind Speed (mph),Hotel Name
1,sur,0,OM,1603676975,64,22.57,59.53,72.27,5.68,Sur Plaza Hotel
15,bhinmal,0,IN,1603677070,52,25.00,72.25,73.26,2.17,Hotel Sai Palace
95,doha,0,QA,1603677074,74,25.29,51.53,78.80,2.24,Al Najada Doha Hotel by Tivoli
138,busselton,0,AU,1603677194,52,-33.65,115.33,73.99,3.00,Broadwater Resort
169,dakar,0,SN,1603677250,88,14.69,-17.44,78.80,8.05,"Radisson Blu Hotel, Dakar Sea Plaza"


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
# Display Map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='500px', margin='0 auto 0 auto', width='500px'))